In [1]:
import os
PATH = os.path.realpath(os.path.dirname(__file__))

# config constant
CONFIG_FILE_PATH_PREFIX = PATH + "/../cfg/"
CONFIG_TMP_FILE_PATH_PREFIX = PATH + "/../cfg/tmp/"
CONFIG_BACKUP_FILE_PATH_PREFIX = PATH + "/../cfg/backup/"

OUTPUT_FILE_PATH_PREFIX = PATH + "/../output/"
OUTPUT_TMP_FILE_PATH_PREFIX = PATH + "/../output/tmp/"

CONFIG_NUMBER = 10000

In [2]:
#!pip install Faker

In [3]:
import sys
import pyspark
from pyspark.sql import SparkSession, Row
from pyspark.sql.types import StructType,StructField, StringType
spark = SparkSession \
    .builder \
    .appName("Python Spark Customer Fact Generator") \
    .getOrCreate()

23/01/05 11:31:14 WARN Utils: Your hostname, htknphuong-virtual-machine resolves to a loopback address: 127.0.1.1; using 192.168.142.129 instead (on interface ens33)
23/01/05 11:31:14 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/01/05 11:31:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/01/05 11:31:15 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
##Reading the config from file for max id
import os.path
idConfig = {
    "accountid": 10000000,
    "defaultaccountid": 10000000,
    "transactionid": 0
}
print('-----------------')
for file in idConfig.keys():
    file_exist = os.path.exists(CONFIG_FILE_PATH_PREFIX+file)
    if (file_exist):
        print(file + ' is exist')
        print('begin loading' + file)
        realFile = open(CONFIG_FILE_PATH_PREFIX+file,"r")
        idConfig[file] = int(realFile.readlines()[0])
        realFile.close()
    else:
        if (file not in ["transactionid"]):
            print ('Please run CustomerFactGenerator first')
            break
        print(file + ' is not exist')
        print('begin to creating ' + file)
        realFile = open(CONFIG_FILE_PATH_PREFIX+file,"w+")
        realFile.seek(0)
        realFile.write(str(idConfig[file]))
        realFile.close()

-----------------
accountid is exist
begin loadingaccountid
defaultaccountid is exist
begin loadingdefaultaccountid
transactionid is exist
begin loadingtransactionid


In [5]:
print(idConfig)

{'accountid': 10014006, 'defaultaccountid': 10000000, 'transactionid': 0}


In [8]:
import random
import faker
from datetime import datetime
import re
class TransactionObject:
    TypeTransactionDim = {
        1: "cash withdrawals",
        2: "checks",
        3: "online payments",
        4: "debitcard charges",
        5: "wire transfers",
        6: "loan payments"
    }
    ConcurrencyDim = {
        1: "VND",
        2: "USD",
        3: "SIG"
    }
    TransactionPointDim = {
        1: "ATM",
        2: "BankingApp",
        3: "PostMachine"
    }
    fake = faker.Faker()

    def __init__(self):
        self.transactionid = idConfig["accountid"]
        idConfig["accountid"] += 1
        self.amount = random.randint(1, 10000)
        self.timestamp = TransactionObject.fake.date()
        __TypeTransactionDim = random.choice(list(TransactionObject.TypeTransactionDim.keys()))
        self.typeTransaction = TransactionObject.TypeTransactionDim[__TypeTransactionDim]
        #__concurrency = random.choice(list(TransactionObject.ConcurrencyDim.keys()))
        #self.concurrency = TransactionObject.ConcurrencyDim[__concurrency]
        self.concurrency = "VND"
        __transactionpoint = random.choice(list(TransactionObject.TransactionPointDim.keys()))
        self.transactionpoint = TransactionObject.TransactionPointDim[__transactionpoint]
        __moneyMoveWay = random.choice([0,1])
        if (__moneyMoveWay == 0):
            self.destinationaccountid = TransactionObject.fake.credit_card_full()
            self.sourceCardID = ""
        else:
            self.destinationaccountid = ""
            self.sourceCardID = TransactionObject.fake.credit_card_full()
        
        self.exchangerate = 1
        self.lattitude = random.uniform(-85, 85)
        self.longtitude = random.uniform(-180, 180)


    def toList(self):
        return [Row(
            self.transactionid,
            self.amount,
            self.timestamp,
            self.typeTransaction,
            self.concurrency,
            self.transactionpoint,
            self.destinationaccountid,
            self.sourceCardID,
            self.exchangerate,
            self.lattitude,
            self.longtitude
            )]


In [9]:
TransactionAccount = []
for x in range(CONFIG_NUMBER):
    Transaction = TransactionObject().toList()
    TransactionAccount = TransactionAccount + Transaction

In [10]:
FactTransactionColumns = StructType([
    StructField("transactionid", StringType(), True),
    StructField("amount", StringType(), True),
    StructField("timestamp", StringType(), True),
    StructField("typeTransaction", StringType(), True),
    StructField("concurrency", StringType(), True),
    StructField("transactionpoint", StringType(), True),
    StructField("destinationaccountid", StringType(), True),
    StructField("sourceCardID", StringType(), True),
    StructField("exchangerate", StringType(), True),
    StructField("lattitude", StringType(), True),
    StructField("longtitude", StringType(), True)
    ])

In [11]:
MainDf = spark.createDataFrame(data=TransactionAccount, schema = FactTransactionColumns)

In [12]:
from datetime import datetime
now = datetime.now()
fileName = 'FactTransaction ' + now.strftime("%Y-%m-%d %H-%M-%S-%f") 

In [13]:
print(fileName)

FactTransaction 2023-01-05 11-31-47-202057


In [14]:
MainDf.write.mode('overwrite').option("header","true").csv(OUTPUT_TMP_FILE_PATH_PREFIX + fileName)

In [15]:
for file in idConfig.keys():
    print('begin to rewrite ' + file)
    realFile = open(CONFIG_TMP_FILE_PATH_PREFIX+file,"w+")
    realFile.seek(0)
    realFile.write(str(idConfig[file]))
    realFile.close()
    print('-----------------')

begin to rewrite accountid
-----------------
begin to rewrite defaultaccountid
-----------------
begin to rewrite transactionid
-----------------


In [16]:
import shutil
for file in idConfig.keys():
    shutil.move(CONFIG_FILE_PATH_PREFIX+file, CONFIG_BACKUP_FILE_PATH_PREFIX+file)
    shutil.move(CONFIG_TMP_FILE_PATH_PREFIX+file, CONFIG_FILE_PATH_PREFIX+file)
shutil.move(OUTPUT_TMP_FILE_PATH_PREFIX + fileName, OUTPUT_FILE_PATH_PREFIX+fileName)

'output/FactTransaction 2023-01-05 11-31-47-202057'